In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

---
## 1. Thu thập dữ liệu từ web Zillow bằng cách parse HTML 
File này gồm có các cột:
- Address: Địa chỉ nhà.
- Price: Gía nhà (là giá của nhà đã bán).
- Bedrooms: số phòng ngủ.
- Bathrooms: số phòng tắm.
- Living Area: Diện tích căn nhà (sqft).
- Heating : Hệ thống sưởi.
- Basement : Tầng hầm(Yes/No).
- Yearbuild: Năm xây dựng.
- Fireplace: Lò sưởi(Yes/No).
- Garage: Nhà xe.


In [38]:
Total_pages=10
req_headers = {
    'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'accept-encoding':'gzip, deflate, sdch, br',
    'accept-language':'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
    'cache-control':'max-age=0',
    'upgrade-insecure-requests':'1',
    'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
courses =pd.DataFrame()
Address_ =[]
Price_=[]
Bedrooms_=[]
Bathrooms_ =[]
LivingArea_ = []
Heating_ =[]
Basement_ =[]
Yearbuild_ =[]
Fireplace_ =[]
Garage_ =[]
with requests.Session() as s:
    page =1
    url = 'https://www.zillow.com/manchester-ct/sold/?searchQueryState=%7B"pagination"%3A%7B%7D%2C"usersSearchTerm"%3A"Manchester%2C%20CT"%2C"mapBounds"%3A%7B"west"%3A-72.62056917797851%2C"east"%3A-72.42659182202148%2C"south"%3A41.72559685174378%2C"north"%3A41.82852106786825%7D%2C"regionSelection"%3A%5B%7B"regionId"%3A398963%2C"regionType"%3A6%7D%5D%2C"isMapVisible"%3Atrue%2C"filterState"%3A%7B"fsba"%3A%7B"value"%3Afalse%7D%2C"fsbo"%3A%7B"value"%3Afalse%7D%2C"nc"%3A%7B"value"%3Afalse%7D%2C"fore"%3A%7B"value"%3Afalse%7D%2C"cmsn"%3A%7B"value"%3Afalse%7D%2C"auc"%3A%7B"value"%3Afalse%7D%2C"pmf"%3A%7B"value"%3Afalse%7D%2C"pf"%3A%7B"value"%3Afalse%7D%2C"rs"%3A%7B"value"%3Atrue%7D%2C"ah"%3A%7B"value"%3Atrue%7D%7D%2C"isListVisible"%3Atrue%2C"mapZoom"%3A13%7D'
    while(page < Total_pages):
        time.sleep(1)
        r = s.get(url, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        detailLink = soup.find_all('a',{'class':'list-card-link list-card-link-top-margin list-card-img'})
        prices = soup.find_all('div',{'class':'list-card-price'})
        for price in prices:
            Price_.append(price.text)
        address = soup.find_all('address',{'class':'list-card-addr'})
        for addr in address:
            Address_.append(addr.text)
        page+=1
        url ='https://www.zillow.com/manchester-nh/sold/' +str(page)+'_p/'
        for link in detailLink:
            detail = s.get(link['href'],headers=req_headers)
            soup1 = BeautifulSoup(detail.content, 'html.parser')
            bbl = soup1.find_all('span',{'class':'ds-bed-bath-living-area'},limit =3) # bed-bad-livingarea
            bed_bad_livingarea =''
            for i in bbl:
                bed_bad_livingarea+= i.text +'\t'
            Bedrooms_.append(bed_bad_livingarea.split('\t')[0])
            Bathrooms_.append(bed_bad_livingarea.split('\t')[1])
            LivingArea_.append(bed_bad_livingarea.split('\t')[2])
            heating = soup1.find("span",string=re.compile("Heating:"))
            if heating is not None:
                heating = heating.find_next_sibling('span').text
                Heating_.append(heating)
            else:
                Heating_.append('None')
            basement = soup1.find("span",string=re.compile("Basement"))
            if basement is not None:
                if ':' in basement.find_next('span').text:
                    basement = basement.find_next('span').text.split(':')[1]
                    Basement_.append(basement)
                else:
                    Basement_.append('None')
            else:
                    Basement_.append('None')
            yearbuild = soup1.find("span",string=re.compile("Condition"))
            if yearbuild is not None:
                if ':' in yearbuild.find_next('span').text:
                    yearbuild = yearbuild.find_next('span').text.split(': ')[1]
                    Yearbuild_.append(yearbuild)
                else:
                    Yearbuild_.append('None')
            else:
                    Yearbuild_.append('None')

            fireplace = soup1.find("h5",string=re.compile("Property details"))
            if fireplace is not None:
                if 'Fireplace:' in fireplace.find_previous('span').text:
                    fireplace = fireplace.find_previous('span').text.split(': ')[1]
                    Fireplace_.append(fireplace)
                else:
                    Fireplace_.append('None')
            else:
                    Fireplace_.append('None')

            garage = soup1.find("span",string=re.compile("Property"))
            if garage is not None:
                if 'Garage spaces:' in garage.find_previous('span').text:
                    garage = garage.find_previous('span').text.split(': ')[1]
                    Garage_.append(garage)
                else:
                    Garage_.append('None')
            else:
                    Garage_.append('None')

courses['Address'] = Address_
courses['Bedrooms'] = Bedrooms_
courses['Bathrooms'] = Bathrooms_
courses['LivingArea'] = LivingArea_
courses['Heating'] = Heating_
courses['Basement'] = Basement_
courses['YearBuild'] = Yearbuild_
courses['Fireplace'] = Fireplace_
courses['Garage'] = Garage_
courses['Price'] = Price_
courses.to_csv('courses.csv')

---
## 2. Tiền xử lý dữ liệu


In [39]:
# Đọc dữ liệu
House_df = pd.read_csv('courses.csv')
House_df.head()

,Unnamed: 0,Address,Bedrooms,Bathrooms,LivingArea,Heating,Basement,YearBuild,Fireplace,Garage,Price
0,0,"216 Walek Farms Rd, Manchester, CT 06040",3 bd,3 ba,"2,556 sqft","Baseboard, Gas",Finished,1995,Yes,2,"$300,000"
1,1,"39 S Hawthorne St, Manchester, CT 06040",3 bd,1 ba,"1,080 sqft","Forced air, Gas",Partially finished,1945,None,0,"$170,000"
2,2,"16 Saddle Hill Rd, Manchester, CT 06040",4 bd,4 ba,"3,832 sqft",Forced air,Finished,2007,Yes,2,"$447,900"
3,3,"99 Pond Ln, Manchester, CT 06042",4 bd,3 ba,"1,852 sqft","Forced air, Oil",NaN,1964,Yes,3,"$299,000"
4,4,"430 E Center St #430, Manchester, CT 06040",1 bd,1.5 ba,"1,074 sqft","Forced air, Gas",None,1976,None,0,"$1,200"
